In [3]:
# Required Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [4]:
# Load datasets
sector_inter = pd.read_csv('22122000_inter.csv', parse_dates=['RowKey']).set_index('RowKey')
future_data = pd.read_csv('future_data2.csv', parse_dates=['RowKey']).set_index('RowKey')

In [5]:
sector_inter.index.names = ['RowKey']

In [6]:

# Splitting the data
train, valid = train_test_split(sector_inter, test_size=0.2, shuffle=False)
test = future_data

In [7]:

# Drop INF_Value for XGBoost training and validation
X_train = train.drop('INF_Value', axis=1)
y_train = train['INF_Value']
X_valid = valid.drop('INF_Value', axis=1)
y_valid = valid['INF_Value']
X_test = test.drop('INF_Value', axis=1)

In [8]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

# Define the range of orders and seasonal orders to search through
p_values = [0, 1, 2]  # AutoRegressive order (p)
d_values = [0, 1]     # Differencing order (d)
q_values = [0, 1, 2]  # Moving Average order (q)
seasonal_periods = [7, 30, 13, 12]  # Different seasonal periods to try

best_aic = float("inf")
best_order = (0, 0, 0)
best_seasonal_order = (0, 0, 0, 0)

# Iterate through all possible combinations
for p in p_values:
    for d in d_values:
        for q in q_values:
            for seasonal_period in seasonal_periods:
                order = (p, d, q)
                seasonal_order = (p, d, q, seasonal_period)
                
                # Fit the SARIMA model with the current order and seasonal order
                sarima_model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
                sarima_result = sarima_model.fit(disp=False)
                
                # Calculate AIC (Akaike Information Criterion)
                current_aic = sarima_result.aic
                
                # Update the best order if the current AIC is lower
                if current_aic < best_aic:
                    best_aic = current_aic
                    best_order = order
                    best_seasonal_order = seasonal_order

# Print the best SARIMA order and seasonal order
print("Best SARIMA Order:", best_order)
print("Best Seasonal Order:", best_seasonal_order)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

Best SARIMA Order: (0, 1, 2)
Best Seasonal Order: (0, 1, 2, 30)


In [9]:
# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)
# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)
sarima_predictions = abs(sarima_predictions)
future_sarima_predictions = abs(future_sarima_predictions)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'
C:\Users\lalit\AppData\Local\Packages\Py

In [10]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [50, 100, 200]
}

# Create the XGBoost model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)



Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}


In [11]:
# Train a new XGBoost model with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)
best_xgb_model.fit(X_train, y_train)

# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)
# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)
# Convert xgb_predictions to a pandas Series with a datetime index
xgb_predictions_series = pd.Series(xgb_predictions, index=y_valid.index)
future_xgb_predictions_series = pd.Series(future_xgb_predictions, index=test.index)

In [31]:

future_ensemble_predictions = 0.75 * future_sarima_predictions + 0.25 * future_xgb_predictions_series

# Ensemble Predictions
ensemble_predictions = 0.75 * sarima_predictions + 0.25 * xgb_predictions_series

In [32]:
# Create DataFrame for Test and Future Predictions
predictions_df = pd.DataFrame({
    'Actual': y_valid,
    'SARIMA_Predictions': sarima_predictions,
    'XGBoost_Predictions': xgb_predictions_series,
    'Ensemble_Predictions': ensemble_predictions
}, index=y_valid.index)

future_predictions_df = pd.DataFrame({
    'SARIMA_Future_Predictions': future_sarima_predictions,
    'XGBoost_Future_Predictions': future_xgb_predictions_series,
    'Ensemble_Future_Predictions': future_ensemble_predictions
}, index=test.index)

In [33]:


import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual values
actual_values = predictions_df['Actual']

# SARIMA predictions
sarima_predictions = predictions_df['SARIMA_Predictions']

# XGBoost predictions
xgboost_predictions = predictions_df['XGBoost_Predictions']

# Ensemble predictions
ensemble_predictions = predictions_df['Ensemble_Predictions']

# Calculate MAPE for each method
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

mape_sarima = mape(actual_values, sarima_predictions)
mape_xgboost = mape(actual_values, xgboost_predictions)
mape_ensemble = mape(actual_values, ensemble_predictions)

# Calculate RMSE for each method
rmse_sarima = np.sqrt(mean_squared_error(actual_values, sarima_predictions))
rmse_xgboost = np.sqrt(mean_squared_error(actual_values, xgboost_predictions))
rmse_ensemble = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

# Calculate MAE for each method
mae_sarima = mean_absolute_error(actual_values, sarima_predictions)
mae_xgboost = mean_absolute_error(actual_values, xgboost_predictions)
mae_ensemble = mean_absolute_error(actual_values, ensemble_predictions)

# Display the results
print("SARIMA MAPE:", mape_sarima)
print("XGBoost MAPE:", mape_xgboost)
print("Ensemble MAPE:", mape_ensemble)

print("SARIMA RMSE:", rmse_sarima)
print("XGBoost RMSE:", rmse_xgboost)
print("Ensemble RMSE:", rmse_ensemble)

print("SARIMA MAE:", mae_sarima)
print("XGBoost MAE:", mae_xgboost)
print("Ensemble MAE:", mae_ensemble)
future_predictions_df
predictions_df

SARIMA MAPE: 5.488642958660144
XGBoost MAPE: 35.79097846913388
Ensemble MAPE: 7.135974803906413
SARIMA RMSE: 65.56765307098286
XGBoost RMSE: 334.0288581247417
Ensemble RMSE: 75.21271623308009
SARIMA MAE: 45.37873273057175
XGBoost MAE: 318.3520122766454
Ensemble MAE: 63.2236349495557


,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-05,855.639981,875.512062,812.929626,859.866453
2022-09-06,855.659981,883.531281,818.183838,867.194420
2022-09-07,877.199980,886.318814,803.634705,865.647787
2022-09-08,868.439981,855.465209,870.250793,859.161605
2022-09-09,842.989981,875.566791,871.968567,874.667235
...,...,...,...,...
2022-12-08,861.619981,975.112148,544.005798,867.335561
2022-12-09,874.389980,970.620607,510.989349,855.712793
2022-12-10,877.479980,965.559431,513.862915,852.635302


In [30]:
#mean of Actual values
mean_actual = predictions_df['Actual'].mean()
mean_actual

881.696949988889

In [34]:
future_predictions_df

,SARIMA_Future_Predictions,XGBoost_Future_Predictions,Ensemble_Future_Predictions
RowKey,,,
2022-12-13,1008.775547,511.633789,884.490108
2022-12-14,1012.869802,515.594482,888.550972
2022-12-15,1009.081417,529.042542,889.071698
2022-12-16,1011.503390,517.874939,888.096277
2022-12-17,1003.542838,496.563995,876.798128
2022-12-18,1007.604629,548.796753,892.902660
2022-12-19,1000.949981,525.060181,881.977531
